# Working with EarthScope Data Services

This exercise combines  

In [ ]:
# ---------- Imports ----------
import __BLANK1__  
from datetime import datetime
from pathlib import Path
import __BLANK2__

# ---------- Create an EarthScope client to get token ----------
client = __BLANK3__


# Fill this with the service endpoint for SAGE data
SAGE_URL = __BLANK4__
# Fill this with the real base path for GAGE data
GAGE_URL = __BLANK5__ 

DATA_DIR = Path("./data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# ---------- Get an EarthScope token ----------
def get_token(token_path: str = "./") -> str:
    """
    Use the SDK to refresh (if needed) and return an access token.
    """
    # Refresh if necessary
    __BLANK6__

    # Retrieve the token string
    token = __BLANK7__
    return token


# ---------- Helper: common auth header ----------
def auth_header(token: str) -> dict:
    # Use a standard Bearer token header (lowercase 'authorization' is fine)
    return __BLANK8__  # e.g., {"authorization": f"Bearer {token}"}


# ---------- Request a miniSEED file from EarthScope's Dataselect web service ----------
# --------------------------------------------------------------------------------------
def download_data(params: dict, data_directory: Path) -> Path | None:
    """
    Sends GET with query parameters and saves the response body to a file.
    Expected params include: net, sta, loc, cha, start (ISO), end (ISO), etc.
    """
    token = get_token()

    header = auth_header(token) 
    
    # Parse the start datetime to build filename parts
    start_dt = datetime.strptime(params["start"], "%Y-%m-%dT%H:%M:%S")
    year = str(start_dt.year)
    # Day-of-year (001-366) for file naming
    doy =  "{:03d}".format(__BLANK10__)  # hint: see how year is extracted from the start

    # Example filename: STA.NET.LOC.CHA.YEAR.DOY.mseed
    file_name = ".".join([
        params["sta"],
        params["net"],
        params["loc"],
        params["cha"],
        year,
        __BLANK11__,
        "mseed"
    ])
    out_path = Path(Path(data_directory) / file_name)

    # Make the request with params and bearer auth
    r = requests.__BLANK12__(
        URL,
        params=__BLANK13__,
        headers=__BLANK14__,
        stream=True
    ) 

    if r.status_code == requests.codes.__BLANK15__:
        with open(out_path, "wb") as f:
            for data in r:
                f.write(data)
    else:
        #problem occured
        print(f"failure: {r.status_code}, {r.reason}")
        return None


# ---------- Requesting data from the EarthScope GAGE web service ----------
# --------------------------------------------------------------------------
def create_url(year: int, day: int, station: str, compression: str) -> str:
    """
    Construct a URL to a file in an archive given year, day-of-year, station, etc.
    Example output:
      {BASE_URL}{year}/{DOY}/{/STATIONDOY0.YY}{compression}
    """
    doy =  "{:03d}".__BLANK16__(day)
    two_digit_year = __BLANK17__  # Hint: use the string slice function in the previous notebook

    file_path = "/".join([str(year), doy])
    file_name = "".join(["/", station, doy, "0.", two_digit_year, compression])
    url = "".join([GAGE_URL, file_path, file_name])
    return url


# ---------- Download a GNSS file from the GAGE web service ----------
def download_file(url: str, data_directory: Path) -> Path | None:

    token = get_token()
    header = __BLANK18__
    file_name = Path(url).name
    out_path = Path(Path(data_directory) / file_name)

    r = __BLANK19__(
        url,
        headers=__BLANK20__,
        stream=True
    )  # )

    if r.status_code == __BLANK21__:
        with open(Path(Path(data_directory) / file_name), 'wb') as f:
            for data in r:
                f.write(data)
    else:
        #problem occured
        print(f"failure: {r.status_code}, {r.reason}")
        return None


# ---------- Demo main ----------

"""
Fill in URL/BASE_URL above and reasonable params here to test.
"""
# Example query parameters (adjust to a valid service)
params = {
    "net": "XN",
    "sta": "ABCD",
    "loc": "00",
    "cha": "BHZ",
    "start": "2024-03-15T12:34:56",
    "end":   "2024-03-15T12:39:56",
    # add more service-specific keys as needed (e.g., 'format': 'mseed')
}

# Try the parameterized request
download_data(params, DATA_DIR)

# Try the URL-building flow
year = 2024
day_of_year = 75
station = "ABCD"
compression_ext = ".gz"  # or ".Z" / "" depending on the archive
url = create_url(year, day_of_year, station, compression_ext)
print(f"Created URL: {url}")
download_file(url, DATA_DIR)